# AES-GCM (Galois/Counter Mode)

AES-GCM is the industry standard for authenticated encryption (AEAD). It provides both confidentiality and data integrity, making it well-suited for modern secure communications.

---

## What It Is

AES-GCM combines two primary functions:

- **Confidentiality:**  
  Encrypts data using AES in counter mode (CTR), which generates a keystream that is XORed with the plaintext.
  
- **Integrity and Authenticity:**  
  Computes a cryptographic tag (using GHASH) over the ciphertext (and optional associated data). The tag ensures that any tampering is detected.

---

## How It Works

1. **Keystream Generation:**  
   AES-CTR creates a keystream using:
   - A secret key (K)
   - A nonce (N), typically 96 bits long
   - An incrementing counter

2. **Encryption:**  
   - The plaintext is XORed with the keystream to generate the ciphertext.
   - Simultaneously, a tag T is generated over the ciphertext plus any additional authenticated data (AAD).

3. **Transmission:**  
   The sender transmits the ciphertext along with the tag T (and optionally the AAD).

4. **Decryption & Verification:**  
   - The receiver recomputes the authentication tag using the shared key K, nonce N, ciphertext, and AAD.
   - If the recalculated tag matches T, the receiver decrypts the ciphertext to recover the original plaintext.
   - If not, the message is rejected as it may have been tampered with.

---

## Communication Flow

```text
Alice                           Bob                          Eve
-----                           ---                          ---
Message P
   |--- Encrypt using AES-GCM with key K and nonce N --->
   |           Produces: Ciphertext C + Tag T         
   |                                                  
   |    (Eve sees C and T, but cannot forge a valid tag)      
   |                                                  
   |--- Bob verifies Tag T using K and N --->
   |      If valid, decrypts to recover P      
```

*Note:* If Eve tampers with the ciphertext, the computed tag will not match, and Bob will reject the message.

---

## Parameters

- **Key (K):**  
  AES key, which can be 128, 192, or 256 bits long.

- **Nonce (IV):**  
  A unique 96-bit value for each encryption instance. Never reuse a nonce with the same key.

- **AAD (Associated Authenticated Data):**  
  Optional data that is authenticated but not encrypted (e.g., protocol headers).

- **Tag:**  
  Typically 16 bytes. The tag is used to verify the integrity and authenticity of the ciphertext and AAD.

---

## Security Considerations

- **Fast & Secure:**  
  AES-GCM is highly efficient, secure, and can be parallelized.

- **Non-reuse of Nonce:**  
  Reusing a nonce with the same key is catastrophic; it can reveal the secret key and compromise all messages.

- **Integrity Enforcement:**  
  If tag verification fails, the entire message must be rejected to avoid data corruption or tampering.

---

## Real-World Applications

- **TLS/HTTPS:**  
  AES-GCM is widely used in TLS (versions 1.2+ and 1.3) to secure web traffic.

- **SSH:**  
  Modern SSH implementations rely on AES-GCM for encryption.

- **VPNs and Disk Encryption:**  
  Used for secure data transmission and storage.

- **Messaging Protocols:**  
  Secure messaging apps leverage AES-GCM to protect messages.

---


**Summary:**  
AES-GCM is the preferred mode for modern cryptographic applications. It efficiently provides both data confidentiality and integrity, and its use is critical in securing web traffic, communications, and storage systems. Proper management of keys and nonces is essential to maintain its security.

In [3]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# --- Key and nonce ---
key = get_random_bytes(16)     # AES-128 (16 bytes); AES-256 needs 32 bytes
nonce = get_random_bytes(12)   # GCM standard = 96-bit nonce

# --- Encrypt ---
cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

plaintext = b"AES-GCM: confidentiality + integrity"
aad = b"header-data"   # associated data (authenticated, not encrypted)

cipher.update(aad)
ciphertext, tag = cipher.encrypt_and_digest(plaintext)

print("Ciphertext (hex):", ciphertext.hex())
print("Tag (hex):", tag.hex())

# --- Decrypt ---
cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce)
cipher_dec.update(aad)

try:
    recovered = cipher_dec.decrypt_and_verify(ciphertext, tag)
    print("Recovered:", recovered)
except ValueError:
    print("Tampering detected, verification failed")


Ciphertext (hex): 19cf4194079449945f4045485d77f0fb227bdc318935b77f3a8f490286bcbf941991265e
Tag (hex): 1cf63a4873142d2d7df1f59a434ce943
Recovered: b'AES-GCM: confidentiality + integrity'
